# CartPole-v1

In [26]:
# 1) Imports
from src.utils.config import load_configs, make_env, make_agent
from src.train import ExperimentRunner, ExperimentResult
import matplotlib.pyplot as plt
from pathlib import Path
import gymnasium as gym
import numpy as np
import torch

ModuleNotFoundError: No module named 'src'

In [ ]:
import sys
from pathlib import Path

# Añade la ruta del proyecto (ajusta si tu notebook está en otra carpeta)
sys.path.append(str(Path().resolve().parent / "src"))

In [27]:
# 2) Create the environment
#    – render_mode='rgb_array' lets you grab frames (e.g. for plotting later)
env = gym.make("CartPole-v1", render_mode="rgb_array")

#### Action and State Space

In [19]:
# 3) Inspect spaces
print("Observation space:", env.observation_space)  # Box(4,)
print("Action space:", env.action_space)            # Discrete(2)

print('Space shape: ', env.observation_space.shape)
print('Action shape: ', env.action_space)


Observation space: Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
Action space: Discrete(2)
Space shape:  (4,)
Action shape:  Discrete(2)


#### Running an Episode of CartPole-V1

In [20]:
# 4) Reset and take a few random steps
obs, info = env.reset(seed=42)
frames = []
for step in range(10):
    action = env.action_space.sample()              # random action: 0 or 1
    obs, reward, done, truncated, info = env.step(action)
    frame  = env.render()
    print(frame.shape)
    frames.append(env.render())
    print(f"Step {step:02d} – obs={obs.round(2)}, reward={reward}, done={done}")
    if done:
        obs, info = env.reset() 

(400, 600, 3)
Step 00 – obs=[ 0.03 -0.2   0.04  0.32], reward=1.0, done=False
(400, 600, 3)
Step 01 – obs=[ 0.02 -0.4   0.04  0.63], reward=1.0, done=False
(400, 600, 3)
Step 02 – obs=[ 0.02 -0.59  0.06  0.93], reward=1.0, done=False
(400, 600, 3)
Step 03 – obs=[ 0.   -0.4   0.07  0.66], reward=1.0, done=False
(400, 600, 3)
Step 04 – obs=[-0.   -0.59  0.09  0.97], reward=1.0, done=False
(400, 600, 3)
Step 05 – obs=[-0.02 -0.79  0.11  1.29], reward=1.0, done=False
(400, 600, 3)
Step 06 – obs=[-0.03 -0.6   0.13  1.03], reward=1.0, done=False
(400, 600, 3)
Step 07 – obs=[-0.04 -0.79  0.15  1.37], reward=1.0, done=False
(400, 600, 3)
Step 08 – obs=[-0.06 -0.6   0.18  1.12], reward=1.0, done=False
(400, 600, 3)
Step 09 – obs=[-0.07 -0.41  0.2   0.89], reward=1.0, done=False


In [21]:
# 5) Close when done
env.close()

In [22]:
frames_np = np.stack(frames)  # Shape: (num_frames, alto, ancho, 3)
frames_tensor = torch.from_numpy(frames_np)  # Tensor de shape igual

# Si quieres que el canal de color sea la segunda dimensión (formato PyTorch: N, C, H, W):
frames_tensor = frames_tensor.permute(0, 3, 1, 2)  # (num_frames, 3, alto, ancho)
    
    


In [25]:
config_dir = Path("configs/")
cfg = load_configs(config_dir)
runner = ExperimentRunner(cfg)
all_results = runner.run()

NameError: name 'Path' is not defined